## Environment Preparation

* Load the library code from the local package directory
* Load the API key and secret in the .env file

In [1]:
# Quick hack to load local library code
import os

os.chdir(os.path.join(os.getcwd(), ".."))

# Load API key and secret from environment variables
from dotenv import load_dotenv
load_dotenv()

True

## ValidMind Python Library Introduction

In [2]:
import pandas as pd
import xgboost as xgb

from numpy import argmax
from sklearn.metrics import accuracy_score, precision_recall_curve
from sklearn.model_selection import train_test_split

%matplotlib inline

## Initializing the ValidMind Library

After creating an account with ValidMind, we can find the project's API key and secret in the settings page of the ValidMind dashboard:

<img src="https://vmai.s3.us-west-1.amazonaws.com/sdk-images/settings.png" width="600" height="300" />

The library credentials can be configured in two ways:

- By setting the `VM_API_KEY` and `VM_API_SECRET` environment variables or
- By passing `api_key` and `api_secret` arguments to the `init` function like this:

```python
vm.init(
    api_key='<your-api-key>',
    api_secret='<your-api-secret>',
    project="cl2r3k1ri000009jweny7ba1g"
)
```

The `project` argument is mandatory since it allows the library to associate all data collected with a specific account project.

In [3]:
import validmind as vm

# For test environment use api_host="https://api.test.vm.validmind.ai/api/v1/tracking"
vm.init(
    # api_host="https://api.dev.vm.validmind.ai/api/v1/tracking",
    # project="cl2r3k1ri000009jweny7ba1g"
    project="cl1jyv16o000809lg98gi9tie"
)

True

### Using a demo dataset

For this simple demonstration, we will use the following bank customer churn dataset from Kaggle: https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data.

We will train a sample model and demonstrate the following library functionalities:

- Logging information about a dataset
- Running data quality tests on a dataset
- Logging information about a model
- Logging training metrics for a model
- Running model evaluation tests

Before we logging any data on a new project, the ValidMind dashboard will let users know that they can automatically populate the different documentation sections by integrating the ValidMind into a model development environment:

<img src="https://vmai.s3.us-west-1.amazonaws.com/sdk-images/empty-data-description.png" width="600" height="300">

#### Logging general project metadata with `log_metadata`

The ValidMind library provides a function to log free-form metadata for a project. A list of preconfigured `content_id` can be used to select where in the dashboard documentation we want this metadata to be displayed. As an example, if a model developer wants to populate the `Model Overview` section for a project, they can use `model_overview` as the `content_id`:

```python
vm.log_metadata("model_overview", text="Testing")
```

The `text` argument accepts Markdown formatted text as we'll see in the cell below. The documentation used for this model has been taken from the [Kaggle dataset](https://www.kaggle.com/code/kmalit/bank-customer-churn-prediction/data).

In [4]:
model_overview = """
Testing writing metadata from the framework
The ValidMind library provides a function to log free-form metadata for a project. A list of preconfigured `content_id` can be used to select where in the dashboard documentation we want this metadata to be displayed. As an example, if a model developer wants to populate the `Model Overview` section for a project, they can use `model_overview` as the `content_id`

We aim to accomplish the following for this study:

- Identify and visualize which factors contribute to customer churn
- Build a prediction model that will perform the following:
  - Classify if a customer is going to churn or not
  - Preferably and based on model performance, choose a model that will attach a probability
  to the churn to make it easier for customer service to target low hanging fruits in their
  efforts to prevent churn
"""

vm.log_metadata(content_id="model_overview", text=model_overview)

Successfully logged metadata


True

The dashboard should now display the `Model Overview` section with the text we have provided from the library:

<img src="https://vmai.s3.us-west-1.amazonaws.com/sdk-images/model-overview.png" width="600" height="300">

#### Analyzing a dataset with `analyze_dataset`

The `analyze_dataset` function will collect the following metadata about the given dataset:

- Field types and descriptions
- Descriptive statistics
- Data distribution histograms
- Feature correlation

and will run a collection of data quality tests such as:

- Class imbalance
- Duplicates
- High cardinality
- Missing values
- Skewness

ValidMind evaluates if the data quality metrics are within expected ranges. These thresholds or ranges can be further configured by model validators.

In [5]:
df = pd.read_csv("notebooks/datasets/bank_customer_churn.csv")

targets = vm.DatasetTargets(
    target_column="Exited",
    class_labels={
        "0": "Did not exit",
        "1": "Exited",
    }
)

vm_dataset = vm.init_dataset(
    dataset=df,
    type="training",
    targets=targets,
)

Pandas dataset detected. Initializing VM Dataset instance...
Inferring dataset types...


In [6]:
vm.analyze_dataset(vm_dataset)

Calculating decriptive statistics...
Preparing in-memory dataset copy...
Calculating feature correlations...
Preparing in-memory dataset copy...
Logging dataset metadata to ValidMind...
Successfully logged dataset metadata and statistics.
Generating correlation plots...


In [7]:
vm.run_dataset_tests(vm_dataset)

Running data quality tests for "training" dataset...

Preparing in-memory dataset copy...


100%|██████████| 6/6 [00:00<00:00, 34.65it/s]


Test suite has completed.
Sending results to ValidMind...


Successfully logged test results for test: class_imbalance
Successfully logged test results for test: duplicates
Successfully logged test results for test: cardinality
Successfully logged test results for test: missing
Successfully logged test results for test: skewness
Successfully logged test results for test: zeros

Summary of results:

Test             Passed      # Passed    # Errors    % Passed
---------------  --------  ----------  ----------  ----------
class_imbalance  True               1           0         100
duplicates       True               1           0         100
cardinality      False              6           1     85.7143
missing          True              14           0         100
skewness         False              6           1     85.7143
zeros            False              0           2           0



[TestResults(category='data_quality', test_name='class_imbalance', params={'min_percent_threshold': 0.2}, passed=True, results=[TestResult(test_name=None, column='Exited', passed=True, values={0: 0.798, 1: 0.202})]),
 TestResults(category='data_quality', test_name='duplicates', params={'min_threshold': 1}, passed=True, results=[TestResult(test_name=None, column=None, passed=True, values={'n_duplicates': 0, 'p_duplicates': 0.0})]),
 TestResults(category='data_quality', test_name='cardinality', params={'num_threshold': 100, 'percent_threshold': 0.1, 'threshold_type': 'percent'}, passed=False, results=[TestResult(test_name=None, column='Surname', passed=False, values={'n_distinct': 2616, 'p_distinct': 0.327}), TestResult(test_name=None, column='Geography', passed=True, values={'n_distinct': 3, 'p_distinct': 0.000375}), TestResult(test_name=None, column='Gender', passed=True, values={'n_distinct': 2, 'p_distinct': 0.00025}), TestResult(test_name=None, column='NumOfProducts', passed=True, v

We can now see all dataset related metadata and data quality metrics in the ValidMind dashboard:

<img src="https://vmai.s3.us-west-1.amazonaws.com/sdk-images/data-description.png" width="600" height="300">

### Preparing the dataset for training

Before we train a model, we need to run some common minimal feature selection and engineering steps on the dataset:

- Dropping irrelevant variables
- Encoding categorical variables

#### Dropping irrelevant variables

The following variables will be dropped from the dataset:

- `RowNumber`: it's a unique identifier to the record
- `CustomerId`: it's a unique identifier to the customer
- `Surname`: no predictive power for this variable
- `CreditScore`: we didn't observer any correlation between `CreditScore` and our target column `Exited`

In [ ]:
df.drop(["RowNumber", "CustomerId", "Surname", "CreditScore"], axis=1, inplace=True)

#### Encoding categorical variables

We will apply one-hot or dummy encoding to the following variables:

- `Geography`: only 3 unique values found in the dataset
- `Gender`: convert from string to integer

In [ ]:
genders = {"Male": 0, "Female": 1}
df.replace({"Gender": genders}, inplace=True)

In [ ]:
df = pd.concat([df, pd.get_dummies(df["Geography"], prefix="Geography")], axis=1)
df.drop("Geography", axis=1, inplace=True)

We are now ready to train our model with the preprocessed dataset:

In [ ]:
df.head()

#### Dataset preparation

For training our model, we will **randomly** split the dataset in 3 parts:

- `training` split with 60% of the rows
- `validation` split with 20% of the rows
- `test` split with 20% of the rows

The `test` dataset will be our held out dataset for model evaluation.

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.20)

# This guarantees a 60/20/20 split
train_ds, val_ds = train_test_split(train_df, test_size=0.25)

# For training
x_train = train_ds.drop("Exited", axis=1)
y_train = train_ds.loc[:, "Exited"].astype(int)
x_val = val_ds.drop("Exited", axis=1)
y_val = val_ds.loc[:, "Exited"].astype(int)

# For testing
x_test = test_df.drop("Exited", axis=1)
y_test = test_df.loc[:, "Exited"].astype(int)

### Model training

We will train a simple XGBoost model and set its `eval_set` to `[(x_train, y_train), (x_val, y_val)]` in order to collect validation datasets metrics on every round. The ValidMind library supports collecting any type of "in training" metrics so model developers can provide additional context to model validators if necessary.

In [ ]:
model = xgb.XGBClassifier(early_stopping_rounds=10)
model.set_params(
    eval_metric=["error", "logloss", "auc"],
)
model.fit(
    x_train,
    y_train,
    eval_set=[(x_train, y_train), (x_val, y_val)],
    verbose=False,
)

# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression()
# model.fit(
#     x_train,
#     y_train,
# )

In [ ]:
y_pred = model.predict_proba(x_val)[:, -1]
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_val, predictions)

print(f"Accuracy: {accuracy}")

#### Evaluating model performance `evaluate_model`

The `evaluate_model` function will extract model metadata and metrics, and will run a collection of model evaluation tests, according to the model type, validation use case and custom validation requirements.

The following model metadata is collected:

- Model framework and architecture (e.g. XGBoost, Random Forest, Logistic Regression, etc.)
- Model task details (e.g. binary classification, regression, etc.)
- Model hyperparameters (e.g. number of trees, max depth, etc.)

The model metrics that are collected depend on the model type, use case, etc. For example, for a binary classification model, the following metrics could be collected (again, depending on configuration):

- AUC
- Error rate
- Logloss
- Feature importance

Similarly, different model evaluation tests are run depending on the model type, use case, etc. For example, for a binary classification model, the following tests could be executed:

- Simple training/test overfit test
- Training/test performance degradation
- Baseline test dataset performance test

In [ ]:
# Find an optimal threshold for the model before we evaluate it
# We want to focus on a threshold that maximizes the F1 score since
# we are interested in optimizing performance for the minority class
y_pred = model.predict_proba(x_val)[:, -1]

precision, recall, thresholds = precision_recall_curve(y_val, y_pred)
fscore = (2 * precision * recall) / (precision + recall)
# Get the index of the largest F1 Score
ix = argmax(fscore)
threshold = thresholds[ix]
print('Optimal threshold=%f, F1 Score=%.3f' % (threshold, fscore[ix]))

In [ ]:
threshold_metric = vm.Metric(
    type="evaluation",
    scope="test",
    key="decision_threshold",
    value=[threshold]
)

vm.log_metrics([threshold_metric])

In [ ]:
eval_results = vm.evaluate_model(
    model,
    train_set=(x_train, y_train),    
    val_set=(x_val, y_val),
    test_set=(x_test, y_test),
    # eval_opts={"decision_threshold": threshold}
)